In [ ]:
import numpy as np
import scipy.ndimage as ndi
from igraph import *
import matplotlib.pyplot as plt
import os
import readimg
from skimage.draw import circle_perimeter
import natsort
from ipywidgets import interact, fixed
import pickle

def getFiles(inFolder, batchName, condStr=''):

	forbiddenFolders = ['ROI', 'maximum_projection', 'equalized']

	if os.name=='nt':
		if inFolder[-1]!='\\':
			if inFolder[-1]=='/':
				properInFolder = inFolder[:-1] + '\\'
				inFolder = inFolder[:-1] + '/'
			else:
				properInFolder = inFolder + '\\'
				inFolder = inFolder + '/'
	else:
		properInFolder = inFolder

	foldersTree = Graph(directed=True)
	foldersTree.add_vertex(name=batchName, fullName=batchName+'/', isFile=0)
	nameMap = {batchName:0}
	dataTree = os.walk(properInFolder+batchName)
	k = 1
	for fullPfolder, folders, files in dataTree:
		if os.name=='nt':
			Pfolder = fullPfolder.split('\\')[-1]
		else:
			Pfolder = fullPfolder.split('/')[-1]
		if Pfolder not in forbiddenFolders:
			sortedFiles = natsort.natsorted(files)
			sortedFolders = natsort.natsorted(folders)
			for file in sortedFiles:
				if condStr in file:
					foldersTree.add_vertex(name=file, fullName=foldersTree.vs[nameMap[Pfolder]]['fullName']+file, isFile=1)
					nameMap[file] = k
					k += 1
					foldersTree.add_edge(nameMap[Pfolder], nameMap[file])
			for folder in sortedFolders:
				if folder not in forbiddenFolders:
					foldersTree.add_vertex(name=folder, fullName=foldersTree.vs[nameMap[Pfolder]]['fullName']+folder+'/', isFile=0)
					nameMap[folder] = k
					k += 1			
					foldersTree.add_edge(nameMap[Pfolder], nameMap[folder])

	files = foldersTree.vs.select(isFile_eq=1)['fullName']
	
	return foldersTree, files

def Graph2Img(g, shape):

	node_pos = g.vs['pos_no']
	ed_pos = g.es['pos_ed']
	img = np.zeros(shape)
	for i in range(len(node_pos)):
		node = node_pos[i]
		if (np.isscalar(node[0])==True):
			img[node[0],node[1],node[2]] = 1
		else:
			for pos in node:
				img[pos[0],pos[1],pos[2]] = 1
			
	for i in range(len(ed_pos)):		
		edge = ed_pos[i]
		for point in edge:
			img[point[0],point[1],point[2]] = 1

	return img

batchName = 'Vascular Project_Picture CD31-PDGFR-vGLUT2-IBA1'

inFolderImg = '/media/Data/cesar/Baptiste data/Project neuroinflammation/'
inFolderBinImg = '../data/Binary/'
inFolderNetImg = '../data/Network/'
inData = '../data/'

dist_thresh = 5  # In micrometers

# Detected microglia
all_blobs = pickle.load(open(f'{inData}Microglia/{batchName}/blobs.dat', 'rb'))

filesTree, files = getFiles(inFolderImg, batchName, '.czi')

In [ ]:
microglia_data = {}
for q in range(len(files)):
    print(q)
    
    splitIndex = files[q].rfind('/')
    filePath, file = files[q][:splitIndex]+'/', files[q][splitIndex+1:]
    dotIndex = file.rfind('.')
    fileName = file[:dotIndex]
    
    blobs = all_blobs[fileName].astype(int)
    
    img_vessel, scale = readimg.ReadImg(inFolderImg+filePath+file, 0)
    
    size_z, size_x, size_y = img_vessel.shape

    binFilePath = f'{inFolderBinImg}{filePath}numpy/{fileName}.npy'
    img_bin = np.load(binFilePath)
        
    netFilePath = f'{inFolderNetImg}{filePath}core/numpy/{fileName}.npy'
    g = np.load(netFilePath, encoding="latin1").item()
    img_g = Graph2Img(g, img_bin.shape)
    
    img_and = np.logical_and(img_bin, img_g)
    
    img_prop = ndi.binary_propagation(img_g, mask=img_bin)  # Reconstruction by dilation
    
    volume = size_z*size_x*size_y*scale[0]*scale[1]*scale[2]*1e-9    
    
    img_dist = ndi.distance_transform_edt(np.logical_not(img_bin), sampling=scale)
    
    blob_dists = []
    blob_touching = []
    for blob in blobs:
        blob_dist = img_dist[tuple(blob[:-1])]
        blob_dists.append(blob_dist)
        
        if blob_dist<=dist_thresh:
            blob_touching.append(True)
        else:
            blob_touching.append(False)
            
    blob_count = blobs.shape[0]           # Number of microglia
    blob_density = blob_count/volume      # density of microglia
    blob_avg_dist = np.mean(blob_dists)   # Distance between microglia and blood vessels
    blob_touching_frac = np.sum(blob_touching)/blob_count   # Microglia touching blood vessels
    
    microglia_data[fileName] = {'blob_count':blob_count, 'blob_density':blob_density, 
                                'blob_avg_dist':blob_avg_dist, 'blob_touching_frac':blob_touching_frac}
    
    pickle.dump(microglia_data, open(f'{inData}Microglia/{batchName}/blobs_props.dat', 'wb'))

In [ ]:
out_file_path = f'../results/{batchName}_microglia.csv'
microglia_data = pickle.load(open(f'{inData}Microglia/{batchName}/blobs_props.dat', 'rb'))
filesTree, files = getFiles(inFolderImg, batchName, '.czi')
files = natsort.natsorted(files)

result_str = f'Name;Microglia density (1/mm^3);Avg. distance to vessel (um);Fraction of microglia touching a vessel\r\n'
for q in range(len(files)):
    
    splitIndex = files[q].rfind('/')
    filePath, file = files[q][:splitIndex]+'/', files[q][splitIndex+1:]
    dotIndex = file.rfind('.')
    fileName = file[:dotIndex]
    
    img_info = microglia_data[fileName]
    
    result_str += f"{fileName};{img_info['blob_density']:.1f};{img_info['blob_avg_dist']*1e3:.1f};{img_info['blob_touching_frac']:.2f}\r\n"
    
open(out_file_path, 'w').write(result_str)
    